In [1]:
import os
import math
import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Dense, Flatten, Lambda, Dropout, Activation, LSTM, GRU, \
        TimeDistributed, Convolution1D, MaxPooling1D, Convolution2D, MaxPooling2D, \
        BatchNormalization, GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, \
        ZeroPadding2D, Reshape, merge, GlobalAveragePooling2D, GlobalMaxPooling2D, AveragePooling2D
from keras.layers.local import LocallyConnected1D
from keras.layers.advanced_activations import ELU
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, CSVLogger, TensorBoard
from keras import backend as K
from keras.models import Model
from keras.models import load_model  
from sklearn.model_selection import train_test_split
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
%matplotlib inline

import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)
KTF.set_session(sess)

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def base_conv_block(num_conv_filters, kernel_size):
    def f(input_):
        x = BatchNormalization()(input_)
        x = Activation('relu')(x)
        out = Convolution2D(num_conv_filters, kernel_size, padding='same')(x)
        return out
    return f

In [3]:
def multi_scale_block(num_conv_filters):
    def f(input_):
        branch1x1 = base_conv_block(num_conv_filters, 1)(input_)
        
        branch3x3 = base_conv_block(num_conv_filters, 1)(input_)  
        branch3x3 = base_conv_block(num_conv_filters, 3)(branch3x3)  
  
        branch5x5 = base_conv_block(num_conv_filters, 1)(input_)  
        branch5x5 = base_conv_block(num_conv_filters, 5)(branch5x5) 
  
        branchpool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same')(input_)  
        branchpool = base_conv_block(num_conv_filters, 1)(branchpool) 
        
        out = concatenate([branch1x1,branch3x3,branch5x5,branchpool], axis=-1)
#         out = base_conv_block(num_conv_filters, 1)(out)
        return out
    return f

In [4]:
def dense_block(num_dense_blocks, num_conv_filters):
    def f(input_):
        x = input_
        for _ in range(num_dense_blocks):
            out = multi_scale_block(num_conv_filters)(x)
            x = concatenate([x, out], axis=-1)
        return x
    return f

In [5]:
def transition_block(num_conv_filters):
    def f(input_):
        x = BatchNormalization()(input_)
        x = Activation('relu')(x)
        x = Convolution2D(num_conv_filters, 1)(x)
        out = AveragePooling2D(pool_size=(2, 2), strides=(2, 2))(x)
        return out
    return f

In [6]:
def multi_scale_level_cnn(input_shape, num_dense_blocks, num_conv_filters, num_classes):
    model_input = Input(shape=input_shape)
    
    x = Convolution2D(num_conv_filters, 3, padding='same')(model_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(4, 1))(x)
    
    x = dense_block(num_dense_blocks, num_conv_filters)(x)
    x = transition_block(num_conv_filters)(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = GlobalAveragePooling2D()(x)
    
    model_output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=model_input, outputs=model_output)
    
    return model

In [7]:
def process_data_for_conv2D(X, resize_shape=None):
    X_conv2D = []
    for sample in X:
        sample = np.reshape(sample, newshape=(sample.shape[0], sample.shape[1], 1))
        if resize_shape:
            sample = resize(sample, output_shape=resize_shape)
        X_conv2D.append(sample)
    return np.array(X_conv2D, dtype=np.float32)

def data_iter(X, y, batch_size):
    num_samples = X.shape[0]
    idx = list(range(num_samples))
    while True:
        for i in range(0, num_samples, batch_size):
            j = idx[i:min(i+batch_size, num_samples)]
            yield X[j, :], y[j, :]

In [8]:
def train_val_test_split(X, y, train_size, val_size, test_size):
    X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, train_size=train_size, stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=test_size/(test_size + val_size), stratify=y_val_test)
    return X_train, y_train, X_val, y_val, X_test, y_test

In [15]:
X_melspec = np.load('/share/音乐分类2/GTZAN/without_split_features/melspec_feature_2048.npy')
y = np.load('/share/音乐分类2/GTZAN/onehot_labels.npy')
X_melspec = process_data_for_conv2D(X_melspec)
print(X_melspec.shape)
print(y.shape)

(1000, 647, 128, 1)
(1000, 10)


In [16]:
#check the architecture of the net
model = multi_scale_level_cnn(input_shape=(X_melspec.shape[1], X_melspec.shape[2], X_melspec.shape[3]), 
                              num_dense_blocks=3, num_conv_filters=32, num_classes=10)
# model = get_multi_level_cnn_model_3(input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]), num_classes=10)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 647, 128, 1)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 647, 128, 32)  320         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 647, 128, 32)  128         conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 647, 128, 32)  0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [17]:
#without data argumatent
k_fold = 10
num_classes = 10

train_size = 0.8
val_size = 0.1
test_size = 0.1

epochs = 100
batch_size = 8
lr = 0.01
file_name0 = 'GTZAN_model.hdf5'
path  = '/share/音乐分类2/GTZAN/2048log/'
csv_name0 = 'GTZAN_csv.csv'
train_loss_record = []
train_acc_record = []
val_loss_record = []
val_acc_record = []
test_loss_record = []
test_acc_record = []
for i in range(k_fold):
    print('Start %d fold training' % (i+1))
    X_train, y_train, X_val, y_val, X_test, y_test = train_val_test_split(X_melspec, y, train_size=train_size, 
                                                                          val_size=val_size, test_size=test_size)
    file_name = 'ExtendeBallroom2048/'+str(i)+'_fold_'+file_name0
#     log_path  = path+str(i)+'_fold_'+'tensorboard_log'
    csv_path  = path+str(i)+'_fold_'+ csv_name0
    lr_change = ReduceLROnPlateau(monitor="loss", factor=0.5, patience=3, min_lr=0.000)
    model_checkpoint = ModelCheckpoint(file_name, monitor='val_acc', save_best_only=True, mode='max')
    early_stopping = EarlyStopping(monitor='loss', min_delta=0.01, patience=10, mode='min')
    csv_logger = CSVLogger(csv_path)
#     tb_cb = TensorBoard(log_dir=log_path, write_images=1, histogram_freq=1)
    callbacks =[lr_change, model_checkpoint, early_stopping,csv_logger]
    opt = Adam(lr=lr)
    model = multi_scale_level_cnn(input_shape=(X_melspec.shape[1], X_melspec.shape[2], X_melspec.shape[3]), 
                              num_dense_blocks=3, num_conv_filters=32, num_classes=num_classes)
    model.compile(
                loss='categorical_crossentropy',
                metrics=['accuracy'],
                optimizer=opt)
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, 
              validation_data=(X_val, y_val), verbose=1,
              callbacks=callbacks)
    model_best = load_model(file_name)
    train_loss, train_acc = model_best.evaluate(X_train, y_train, batch_size=batch_size, verbose=0)
    val_loss, val_acc = model_best.evaluate(X_val, y_val, batch_size=batch_size, verbose=0)
    test_loss, test_acc = model_best.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
    
    train_loss_record.append(train_loss)
    train_acc_record.append(train_acc)
    val_loss_record.append(val_loss)
    val_acc_record.append(val_acc)
    test_loss_record.append(test_loss)
    test_acc_record.append(test_acc)
    print('\n\n%d fold train loss %.4f train acc %.4f, val loss %.4f val acc %.4f, test loss %.4f test acc %.4f\n\n' % 
          (i+1, train_loss, train_acc, val_loss, val_acc, test_loss, test_acc))
train_loss_avg = np.mean(np.array(train_loss_record))
train_acc_avg = np.mean(np.array(train_acc_record))
val_loss_avg = np.mean(np.array(val_loss_record))
val_acc_avg = np.mean(np.array(val_acc_record))
test_loss_avg = np.mean(np.array(test_loss_record))
test_acc_avg = np.mean(np.array(test_acc_record))
print('\n\n%d fold train loss avg %.4f train acc avg %.4f, val loss avg %.4f val acc avg %.4f, test loss avg %.4f test acc avg %.4f' % 
  (k_fold, train_loss_avg, train_acc_avg, val_loss_avg, val_acc_avg, test_loss_avg, test_acc_avg))

Start 1 fold training
Train on 800 samples, validate on 100 samples
Epoch 1/100
800/800 [==============================] - 31s - loss: 1.8474 - acc: 0.3175 - val_loss: 5.1491 - val_acc: 0.2300
Epoch 2/100
800/800 [==============================] - 27s - loss: 1.5454 - acc: 0.4213 - val_loss: 3.8600 - val_acc: 0.1800
Epoch 3/100
800/800 [==============================] - 27s - loss: 1.4640 - acc: 0.4875 - val_loss: 6.5657 - val_acc: 0.1200
Epoch 4/100
800/800 [==============================] - 28s - loss: 1.4318 - acc: 0.4937 - val_loss: 6.1418 - val_acc: 0.2600
Epoch 5/100
800/800 [==============================] - 27s - loss: 1.3019 - acc: 0.5413 - val_loss: 3.9806 - val_acc: 0.1700
Epoch 6/100
800/800 [==============================] - 28s - loss: 1.2425 - acc: 0.5550 - val_loss: 1.5534 - val_acc: 0.5300
Epoch 7/100
800/800 [==============================] - 27s - loss: 1.2197 - acc: 0.5687 - val_loss: 1.5622 - val_acc: 0.3800
Epoch 8/100
800/800 [==============================] - 27

800/800 [==============================] - 28s - loss: 0.1615 - acc: 0.9488 - val_loss: 0.9909 - val_acc: 0.7900
Epoch 66/100
800/800 [==============================] - 28s - loss: 0.1894 - acc: 0.9350 - val_loss: 0.9472 - val_acc: 0.7400
Epoch 67/100
800/800 [==============================] - 28s - loss: 0.1366 - acc: 0.9613 - val_loss: 0.8919 - val_acc: 0.7900
Epoch 68/100
800/800 [==============================] - 28s - loss: 0.1108 - acc: 0.9738 - val_loss: 0.7290 - val_acc: 0.8300
Epoch 69/100
800/800 [==============================] - 28s - loss: 0.0780 - acc: 0.9800 - val_loss: 0.4829 - val_acc: 0.9000
Epoch 70/100
800/800 [==============================] - 28s - loss: 0.0948 - acc: 0.9750 - val_loss: 0.6308 - val_acc: 0.8400
Epoch 71/100
800/800 [==============================] - 28s - loss: 0.0877 - acc: 0.9750 - val_loss: 0.5625 - val_acc: 0.8800
Epoch 72/100
800/800 [==============================] - 28s - loss: 0.1115 - acc: 0.9675 - val_loss: 0.6076 - val_acc: 0.8800
Epoch

800/800 [==============================] - 28s - loss: 0.5359 - acc: 0.8187 - val_loss: 0.8239 - val_acc: 0.7300
Epoch 37/100
800/800 [==============================] - 28s - loss: 0.4638 - acc: 0.8213 - val_loss: 1.6390 - val_acc: 0.6600
Epoch 38/100
800/800 [==============================] - 28s - loss: 0.4860 - acc: 0.8312 - val_loss: 0.7412 - val_acc: 0.8100
Epoch 39/100
800/800 [==============================] - 28s - loss: 0.5190 - acc: 0.8238 - val_loss: 1.5395 - val_acc: 0.5900
Epoch 40/100
800/800 [==============================] - 28s - loss: 0.3554 - acc: 0.8825 - val_loss: 0.4626 - val_acc: 0.8300
Epoch 41/100
800/800 [==============================] - 28s - loss: 0.3574 - acc: 0.8825 - val_loss: 0.8020 - val_acc: 0.7500
Epoch 42/100
800/800 [==============================] - 27s - loss: 0.3310 - acc: 0.8962 - val_loss: 0.5351 - val_acc: 0.8000
Epoch 43/100
800/800 [==============================] - 28s - loss: 0.3354 - acc: 0.8937 - val_loss: 0.5114 - val_acc: 0.8200
Epoch

800/800 [==============================] - 28s - loss: 0.8706 - acc: 0.7250 - val_loss: 0.9905 - val_acc: 0.6400
Epoch 14/100
800/800 [==============================] - 28s - loss: 0.9343 - acc: 0.6663 - val_loss: 1.1688 - val_acc: 0.5900
Epoch 15/100
800/800 [==============================] - 28s - loss: 0.8834 - acc: 0.7000 - val_loss: 0.8638 - val_acc: 0.6800
Epoch 16/100
800/800 [==============================] - 28s - loss: 0.8204 - acc: 0.7362 - val_loss: 2.2886 - val_acc: 0.3800
Epoch 17/100
800/800 [==============================] - 28s - loss: 0.8529 - acc: 0.6875 - val_loss: 2.3181 - val_acc: 0.4100
Epoch 18/100
800/800 [==============================] - 28s - loss: 0.7923 - acc: 0.7200 - val_loss: 0.7714 - val_acc: 0.7400
Epoch 19/100
800/800 [==============================] - 28s - loss: 0.8199 - acc: 0.7225 - val_loss: 1.1212 - val_acc: 0.6300
Epoch 20/100
800/800 [==============================] - 28s - loss: 0.7672 - acc: 0.7325 - val_loss: 1.0161 - val_acc: 0.6400
Epoch

800/800 [==============================] - 28s - loss: 0.0493 - acc: 0.9862 - val_loss: 0.2113 - val_acc: 0.9200
Epoch 79/100
800/800 [==============================] - 28s - loss: 0.0517 - acc: 0.9900 - val_loss: 0.1946 - val_acc: 0.9300
Epoch 80/100
800/800 [==============================] - 28s - loss: 0.0572 - acc: 0.9825 - val_loss: 0.3052 - val_acc: 0.9000
Epoch 81/100
800/800 [==============================] - 28s - loss: 0.0252 - acc: 0.9963 - val_loss: 0.1698 - val_acc: 0.9200
Epoch 82/100
800/800 [==============================] - 28s - loss: 0.0382 - acc: 0.9887 - val_loss: 0.2574 - val_acc: 0.9200
Epoch 83/100
800/800 [==============================] - 28s - loss: 0.0243 - acc: 1.0000 - val_loss: 0.1988 - val_acc: 0.9300
Epoch 84/100
800/800 [==============================] - 28s - loss: 0.0403 - acc: 0.9950 - val_loss: 0.2355 - val_acc: 0.9200
Epoch 85/100
800/800 [==============================] - 28s - loss: 0.0416 - acc: 0.9938 - val_loss: 0.1979 - val_acc: 0.9200
Epoch

...

800/800 [==============================] - 28s - loss: 0.0512 - acc: 0.9887 - val_loss: 0.6867 - val_acc: 0.8400
Epoch 81/100
800/800 [==============================] - 28s - loss: 0.0904 - acc: 0.9725 - val_loss: 0.7592 - val_acc: 0.8400
Epoch 82/100
800/800 [==============================] - 28s - loss: 0.1290 - acc: 0.9675 - val_loss: 0.8514 - val_acc: 0.8500
Epoch 83/100
800/800 [==============================] - 28s - loss: 0.0454 - acc: 0.9912 - val_loss: 0.6612 - val_acc: 0.8500
Epoch 84/100
800/800 [==============================] - 28s - loss: 0.0415 - acc: 0.9912 - val_loss: 0.6546 - val_acc: 0.8600
Epoch 85/100
800/800 [==============================] - 28s - loss: 0.0472 - acc: 0.9925 - val_loss: 0.6513 - val_acc: 0.8900
Epoch 86/100
800/800 [==============================] - 28s - loss: 0.0472 - acc: 0.9887 - val_loss: 0.6661 - val_acc: 0.8500
Epoch 87/100
800/800 [==============================] - 28s - loss: 0.0334 - acc: 0.9963 - val_loss: 0.6457 - val_acc: 0.8500
Epoch

800/800 [==============================] - 28s - loss: 0.3899 - acc: 0.8688 - val_loss: 1.7098 - val_acc: 0.6600
Epoch 46/100
800/800 [==============================] - 28s - loss: 0.3827 - acc: 0.8775 - val_loss: 1.4837 - val_acc: 0.5800
Epoch 47/100
800/800 [==============================] - 28s - loss: 0.3146 - acc: 0.9125 - val_loss: 1.5859 - val_acc: 0.6200
Epoch 48/100
800/800 [==============================] - 28s - loss: 0.3685 - acc: 0.8838 - val_loss: 0.6040 - val_acc: 0.8400
Epoch 49/100
800/800 [==============================] - 28s - loss: 0.3115 - acc: 0.9087 - val_loss: 1.2314 - val_acc: 0.6800
Epoch 50/100
800/800 [==============================] - 28s - loss: 0.2772 - acc: 0.9113 - val_loss: 1.5657 - val_acc: 0.5900
Epoch 51/100
800/800 [==============================] - 28s - loss: 0.2500 - acc: 0.9262 - val_loss: 0.7566 - val_acc: 0.7500
Epoch 52/100
800/800 [==============================] - 28s - loss: 0.3339 - acc: 0.9050 - val_loss: 3.3267 - val_acc: 0.5100
Epoch

800/800 [==============================] - 28s - loss: 0.9903 - acc: 0.6613 - val_loss: 3.2134 - val_acc: 0.2400
Epoch 13/100
800/800 [==============================] - 28s - loss: 1.0162 - acc: 0.6287 - val_loss: 1.6191 - val_acc: 0.5300
Epoch 14/100
800/800 [==============================] - 27s - loss: 0.9682 - acc: 0.6725 - val_loss: 1.9072 - val_acc: 0.4900
Epoch 15/100
800/800 [==============================] - 28s - loss: 0.9678 - acc: 0.6437 - val_loss: 0.9802 - val_acc: 0.6200
Epoch 16/100
800/800 [==============================] - 28s - loss: 0.8681 - acc: 0.7150 - val_loss: 2.4651 - val_acc: 0.4700
Epoch 17/100
800/800 [==============================] - 28s - loss: 0.9297 - acc: 0.6775 - val_loss: 2.2993 - val_acc: 0.3600
Epoch 18/100
800/800 [==============================] - 28s - loss: 0.9067 - acc: 0.6787 - val_loss: 1.8032 - val_acc: 0.4000
Epoch 19/100
800/800 [==============================] - 28s - loss: 0.8989 - acc: 0.6663 - val_loss: 0.7564 - val_acc: 0.7100
Epoch

800/800 [==============================] - 28s - loss: 0.1072 - acc: 0.9725 - val_loss: 0.3079 - val_acc: 0.8900
Epoch 78/100
800/800 [==============================] - 28s - loss: 0.0840 - acc: 0.9850 - val_loss: 0.2554 - val_acc: 0.9400
Epoch 79/100
800/800 [==============================] - 28s - loss: 0.0711 - acc: 0.9862 - val_loss: 0.2122 - val_acc: 0.9400
Epoch 80/100
800/800 [==============================] - 28s - loss: 0.0658 - acc: 0.9900 - val_loss: 0.1827 - val_acc: 0.9600
Epoch 81/100
800/800 [==============================] - 28s - loss: 0.0773 - acc: 0.9813 - val_loss: 0.2060 - val_acc: 0.9100
Epoch 82/100
800/800 [==============================] - 28s - loss: 0.0651 - acc: 0.9887 - val_loss: 0.1867 - val_acc: 0.9500
Epoch 83/100
800/800 [==============================] - 28s - loss: 0.0561 - acc: 0.9900 - val_loss: 0.1946 - val_acc: 0.9400
Epoch 84/100
800/800 [==============================] - 28s - loss: 0.0540 - acc: 0.9925 - val_loss: 0.2919 - val_acc: 0.9200
Epoch

800/800 [==============================] - 28s - loss: 0.4669 - acc: 0.8375 - val_loss: 1.1992 - val_acc: 0.7100
Epoch 42/100
800/800 [==============================] - 28s - loss: 0.4901 - acc: 0.8400 - val_loss: 0.7086 - val_acc: 0.7400
Epoch 43/100
800/800 [==============================] - 28s - loss: 0.4146 - acc: 0.8650 - val_loss: 1.0552 - val_acc: 0.6100
Epoch 44/100
800/800 [==============================] - 28s - loss: 0.3685 - acc: 0.8775 - val_loss: 1.4841 - val_acc: 0.6400
Epoch 45/100
800/800 [==============================] - 28s - loss: 0.3864 - acc: 0.8637 - val_loss: 1.2931 - val_acc: 0.6700
Epoch 46/100
800/800 [==============================] - 28s - loss: 0.4166 - acc: 0.8600 - val_loss: 0.6703 - val_acc: 0.7500
Epoch 47/100
800/800 [==============================] - 29s - loss: 0.4413 - acc: 0.8500 - val_loss: 2.3566 - val_acc: 0.4100
Epoch 48/100
800/800 [==============================] - 31s - loss: 0.4176 - acc: 0.8538 - val_loss: 0.5509 - val_acc: 0.8100
Epoch

In [ ]:

num_classes = 13

train_size = 0.8
val_size = 0.1
test_size = 0.1

epochs = 100
batch_size = 8
lr = 0.01
train_loss_record = []
train_acc_record = []
val_loss_record = []
val_acc_record = []
test_loss_record = []
test_acc_record = []
for i in range(k_fold):
    print('Start %d fold training' % (i+1))
    X_train, y_train, X_val, y_val, X_test, y_test = train_val_test_split(X_melspec, y, train_size=train_size, 
                                                                          val_size=val_size, test_size=test_size)
    model_best = load_model(file_name)
    train_loss, train_acc = model_best.evaluate(X_train, y_train,batch_size=batch_size,verbose=0)
    val_loss, val_acc = model_best.evaluate(X_val, y_val, batch_size=batch_size,verbose=0)
    test_loss, test_acc = model_best.evaluate(X_test, y_test,batch_size=batch_size, verbose=0)
    
    train_loss_record.append(train_loss)
    train_acc_record.append(train_acc)
    val_loss_record.append(val_loss)
    val_acc_record.append(val_acc)
    test_loss_record.append(test_loss)
    test_acc_record.append(test_acc)
    print('\n\n%d fold train loss %.4f train acc %.4f, val loss %.4f val acc %.4f, test loss %.4f test acc %.4f\n\n' % 
          (i+1, train_loss, train_acc, val_loss, val_acc, test_loss, test_acc))

train_loss_avg = np.mean(np.array(train_loss_record))
train_acc_avg = np.mean(np.array(train_acc_record))
val_loss_avg = np.mean(np.array(val_loss_record))
val_acc_avg = np.mean(np.array(val_acc_record))
test_loss_avg = np.mean(np.array(test_loss_record))
test_acc_avg = np.mean(np.array(test_acc_record))
print('\n\n%d fold train loss avg %.4f train acc avg %.4f, val loss avg %.4f val acc avg %.4f, test loss avg %.4f test acc avg %.4f' % 
  (k_fold, train_loss_avg, train_acc_avg, val_loss_avg, val_acc_avg, test_loss_avg, test_acc_avg))

In [ ]:
model = multi_scale_level_cnn(input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]), 
                              num_dense_blocks=3, num_conv_filters=32, num_classes=10)
model.summary()


epochs = 100
batch_size = 8
opt = Adam(lr=0.0001)
lr_change = ReduceLROnPlateau(monitor="loss", factor=0.5, patience=2, min_lr=0.000)
train_data_iter = data_iter(X_train, y_train, batch_size)
test_data_iter = data_iter(X_test, y_test, batch_size)
model.compile(
            loss='categorical_crossentropy',
            metrics=['accuracy'],
            optimizer=opt)

for e in range(epochs):
    batchs = 0
    for X_batch, y_batch in train_data_iter:
        model.train_on_batch(X_batch, y_batch)
        batchs += 1
        if batchs >= len(X_train) / 32:
            break
    train_evaluation = model.evaluate(X_train, y_train, verbose=0)
    val_evaluation = model.evaluate(X_val, y_val, verbose=0)
    test_evaluation = model.evaluate(X_test, y_test, verbose=0)
    
    print('Epoch %d train_loss: %.4f train_acc: %.4f, val_loss: %.4f val_acc: %.4f, test_loss: %.4f, test_acc: %.4f' % 
          (e+1, train_evaluation[0], train_evaluation[1], val_evaluation[0], val_evaluation[1], test_evaluation[0], test_evaluation[1]))